In [ ]:
import sys
from time import time

def GetStr():
    str1 = input("Input str1: ")
    str2 = input("Input str2: ")
    return str1, str2

def OutputTable(table, str1, str2):
    print("\n   ", end = " ")
    for i in str2:
        print(i, end = " ")

    for i in range(len(table)):
        if i:
            print("\n" + str1[i-1], end = " ")
        else:
            print("\n ", end = " ")
        for j in range(len(table[i])):
            print(table[i][j], end = " ")
    print("\n")

def levenshtein_rec(s1, s2, memory):
    if(len(s1) == 0):
        memory += sys.getsizeof(s1) + sys.getsizeof(s2)
        return len(s2), memory
    
    if(len(s2) == 0):
        memory += sys.getsizeof(s1) + sys.getsizeof(s2)
        return len(s1), memory
    
    memory += sys.getsizeof(1)
    return min(levenshtein_rec(s1, s2[:-1], memory)[0] + 1, 
              levenshtein_rec(s1[:-1], s2, memory)[0] + 1, 
              levenshtein_rec(s1[:-1], s2[:-1], memory)[0] + (0 if (s1[-1] == s2[-1]) else 1)), memory


def levenshtein_damerau_rec(s1, s2, memory):
    if (len(s1) == 0):
        memory += sys.getsizeof(s1) + sys.getsizeof(s2)
        return len(s2), memory

    if (len(s2) == 0):
        memory += sys.getsizeof(s1) + sys.getsizeof(s2)
        return len(s1), memory

    res = min(levenshtein_damerau_rec(s1[:-1], s2, memory)[0], 
              levenshtein_damerau_rec(s1, s2[:-1], memory)[0] + 1,
    levenshtein_damerau_rec(s1[:-1], s2[:-1], memory)[0] + (0 if (s1[-1] == s2[-1]) else 1))
    
    memory += 2 * sys.getsizeof(1)
    
    if (len(s1) > 1 and len(s2) > 1 and s1[-1] == s2[-2] and s1[-2] == s2[-1]):
        res = min(res, levenshtein_damerau_rec(s1[:-2], s2[:-2], memory)[0] + 1)

    return res, memory
    

def levenshtein_matrix(str1, str2, output):
    len_i = len(str1) + 1
    len_j = len(str2) + 1
    table = [[i + j for j in range(len_j)] for i in range(len_i)]
    
    memory = sys.getsizeof(table) + 2 * sys.getsizeof(1)
    
    for i in range(1, len_i):
        for j in range(1, len_j):
            forfeit = 0 if (str1[i - 1] == str2[j - 1]) else 1
            table[i][j] = min(table[i - 1][j] + 1,
                              table[i][j - 1] + 1,
                              table[i - 1][j - 1] + forfeit)
            
    if (output):
        OutputTable(table, str1, str2)
    return(table[-1][-1], memory)

def levenshtein_damerau_matrix(str1, str2, output):
    len_i = len(str1) + 1
    len_j = len(str2) + 1
    table = [[i + j for j in range(len_j)] for i in range(len_i)]
    memory = sys.getsizeof(table) + 2 * sys.getsizeof(1)
    
    for i in range(1, len_i):
        for j in range(1, len_j):
            forfeit = 0 if (str1[i - 1] == str2[j - 1]) else 1
            table[i][j] = min(table[i - 1][j] + 1,
                              table[i][j - 1] + 1,
                              table[i - 1][j - 1] + forfeit)
            if (i > 1 and j > 1) and str1[i-1] == str2[j-2] and str1[i-2] == str2[j-1]:
                table[i][j] = min(table[i][j], table[i-2][j-2] + 1)
    if (output):        
        OutputTable(table, str1, str2)
    return(table[-1][-1]), memory

def count_time(s1, s2):
    t1 = 0 
    t2 = 0 
    t3 = 0 
    t4 = 0
    for i in range(10):
        memory = 0
        
        time1_in = time()
        for i in range(100):
            steps1 = levenshtein_rec(s1, s2, memory)
        time1_out = time()
        
        print ("\nTime (levenshtein_rec): ", (time1_out - time1_in) / 100, "\n")
        t1 += (time1_out - time1_in) / 100

        time2_in = time()
        for i in range(100):
            steps2 = levenshtein_matrix(s1, s2, memory);
        time2_out = time()
        
        print ("\nTime (matrix): ", (time1_out - time1_in) / 100, "\n")
        t2 += (time2_out - time2_in) / 100

        time3_in = time()
        for i in range(100):
            steps3 = levenshtein_damerau_rec(s1, s2, memory)
        time3_out = time()
        
        print ("\nTime (levenshtein_damerau_rec):", (time1_out - time1_in) / 100, "\n")
               
        t3 += (time3_out - time3_in) / 100

        time4_in = time()
        for i in range(100):
            steps4 = levenshtein_damerau_matrix(s1, s2, memory)
        time4_out = time()
        print ("\nTime (levenshtein_damerau_matrix):", (time4_out - time4_in) / 100, "\n")
        t4 += (time4_out - time4_in) / 100
        
    print()
    print("T1", t1/10)
    print("T2", t2/10)
    print("T3", t3/10)
    print("T4", t4/10)


def count_memory(s1, s2):
    memory1 = 0;
    memory2 = 0;
    memory3 = 0;
    memory4 = 0;
    memory1 = levenshtein_rec(s1, s2, memory1)[1]
    memory2 = levenshtein_matrix(s1, s2, memory2)[1]
    memory3 = levenshtein_damerau_rec(s1, s2, memory3)[1]
    memory4 = levenshtein_damerau_matrix(s1, s2, memory4)[1]
    print()
    print("M1: ", memory1, "\n")
    print("M2: ", memory2, "\n")
    print("M3: ", memory3, "\n")
    print("M4: ", memory4, "\n")


def Menu():
    case = 999
    while(case != '0'):
        case = input("Menu:\n \
\t1. Levenshtein distance recursion\n \
\t2. Levenshtein distance table\n \
\t3. Damerau–Levenshtein distance recursion\n \
\t4. Damerau–Levenshtein distance table\n \
\t5. Time analysis\n \
\t6. Memory analysis\n \
\t0. Exit\n")
        if (case == "1"):
            s1, s2 = GetStr()
            levenshtein_rec(s1, s2, 0)
        elif (case == "2"):
            s1, s2 = GetStr()
            levenshtein_matrix(s1, s2, True)
        elif (case == "3"):
            s1, s2 = GetStr()
            (s1, s2, 0)
        elif (case == "4"):
            s1, s2 = GetStr()
            levenshtein_damerau_matrix(s1, s2, True)
        elif (case == "5"):
            count_time("asas", "fead")
        elif (case == "6"):
            count_memory("fdaw", "aewfewa")
       
        else:
            flagDo = False
            
            
if __name__ == "__main__": 
    Menu()